# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Phạm Nguyễn Quốc Thanh
- MSSV: 21127428
- Lớp: 21CLC08

# Import

In [13]:
import pandas as pd
import numpy as np
from IPython.display import Markdown as md

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [14]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)
X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)
Salary_train = train.iloc[:, [-1]].to_numpy() 
Salary_test = test.iloc[:, [-1]]
first_11_data_train = train.iloc[:, :11].to_numpy()
first_11_data_test = test.iloc[:, :11].to_numpy()
first_3_data = train.iloc[:3, :3]
# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [15]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)

def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel())) 

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [16]:
# Phần code cho yêu cầu 1a
lr_1a = OLSLinearRegression().fit(first_11_data_train, y_train.to_numpy())
# required_1a = OLSLinearRegression().fit(first_11_data_test, y_test.to_numpy())
# required_1a.get_params()



In [17]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_hat = lr_1a.predict(first_11_data_test)
print("MAE: ", mae(y_test.to_numpy(), y_hat))
np.set_printoptions(suppress=True, precision=3)
weights = lr_1a.get_params()
print("Weights: ", weights)

MAE:  104863.77754032997
Weights:  [-22756.513    804.503   1294.655 -91781.898  23182.389   1437.549
  -8570.662    147.858    152.888    117.222  34552.286]


In [18]:
first_11_columns = train.columns[:11].to_numpy()
results = zip(first_11_columns, weights)

display(md("Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\\to $ 0.012)"))
display(md("Salary = " + " + ".join(f"{w:.3f} * {f}" for f, w in results)))

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to $ 0.012)

Salary = -22756.513 * Gender + 804.503 * 10percentage + 1294.655 * 12percentage + -91781.898 * CollegeTier + 23182.389 * Degree + 1437.549 * collegeGPA + -8570.662 * CollegeCityTier + 147.858 * English + 152.888 * Logical + 117.222 * Quant + 34552.286 * Domain

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = \text{Gender} \times -31888.616 + \text{10percentage} \times 1947.808 + \text{12percentage} \times 100.178 + \text{CollegeTier} \times -72462.209 + \text{Degree} \times -11070.036 + \text{collegeGPA} \times 851.434 + \text{CollegeCityTier} \times -6642.376 + \text{English} \times 70.451 + \text{Logical} \times -95.119 + \text{Quant} \times 379.677 + \text{Domain} \times 41249.229 $$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [84]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
X_train_1b_all = train.iloc[:, -6:].to_numpy()
np.random.shuffle(X_train_1b_all)

k_fold = 5
div, remain = divmod(X_train_1b_all.shape[0], k_fold)
range_list = []
for i in range(k_fold):
    if i == k_fold - 1:
        range_list.append((i * div, (i + 1) * div + remain))
    else:
        range_list.append((i * div, (i + 1) * div))

for i in range(5):
    total_MAE = 0
    idx = 19
    for fold in range_list:
        lr_1b = OLSLinearRegression().fit(X_train_1b_all[fold[0]:fold[1], i].reshape(-1, 1), X_train_1b_all[fold[0]:fold[1], -1].reshape(-1, 1))
        y_pred = lr_1b.predict(X_train.iloc[:,idx].to_numpy().reshape(-1, 1))
        total_MAE += mae(y_train.to_numpy(),y_pred)
    idx -= 1
    print(f"Feature {i+1}: {total_MAE/5}")

Feature 1: 309827.77597711724
Feature 2: 300724.4929218789
Feature 3: 307485.7699296508
Feature 4: 315092.553378702
Feature 5: 312000.6470063998


In [20]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

In [21]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [22]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

In [23]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

In [24]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [25]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [26]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [27]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [28]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$